##### Local Environment Setup

In [1]:

import os
import sys
# Set JAVA env variable
os.environ["JAVA_HOME"] = r"C:\Program Files\Eclipse Adoptium\jdk-11.0.26.4-hotspot"
# Set Hadoop environment variables 
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = os.environ['HADOOP_HOME'] + r'\bin;' + os.environ['PATH']
# Set the Python executable path explicitly
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Get Pipeline Tools Path
sys.path.append(R'C:\GitHub\Tools\de')


##### Libraries

In [2]:
import time
import logging
from datetime import datetime

import numpy as np
import pandas as pd

from typing import Dict


from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import (StructType, StructField, StringType, 
                            DoubleType, IntegerType, TimestampType, 
                            DateType)
from delta.tables import DeltaTable


In [3]:
from de_pipeline_tools import *

##### Spark Session

In [4]:
spark = initialize_local_spark_delta_lake("Financial Data Pipeline")

2025-04-09 16:10:02,467 - INFO - ---Spark session initialized with Delta Lake support---


In [5]:
# Create a database in the Hive warehouse if doesn't exist
spark.sql("CREATE DATABASE IF NOT EXISTS de_pipelines LOCATION 'C:/hive-warehouse/de_pipelines'")

DataFrame[]

##### Inputs/Outputs

In [6]:
data_dir = "../../data"
source_path     = f"{data_dir}/financial_transactions.csv"

source_path  = os.path.abspath(os.path.join(os.getcwd(), *source_path.split('/')))

bronze_table = 'de_pipelines.financial_osb_bronze'
silver_table = 'de_pipelines.financial_osb_silver'
gold_table   = 'de_pipelines.financial_osb_gold'


In [7]:
def get_schema():
    return StructType([
        StructField("transaction_id", StringType(), True),
        StructField("timestamp", StringType(), True), # need to process as StringType and convert to Timestamp
        StructField("customer_id", StringType(), True),
        StructField("account_number", StringType(), True),
        StructField("transaction_type", StringType(), True),
        StructField("amount", DoubleType(), True),
        StructField("currency", StringType(), True),
        StructField("balance_after", DoubleType(), True),
        StructField("status", StringType(), True),
        StructField("merchant", StringType(), True),
        StructField("category", StringType(), True),
        StructField("location", StringType(), True)
    ])

### Define Transformation Functions and Validation Rules

In [8]:
# Define bronze validation rules
bronze_validation_rules = [
    {
        "name": "has_transaction_id",
        "condition": "transaction_id IS NOT NULL",
        "description": "Transaction ID must be present"
    },
    {
        "name": "valid_amount",
        "condition": "amount IS NOT NULL AND amount > 0",
        "description": "Amount must be positive if not null"
    },
    {
        "name": "valid_timestamp",
        "condition": "timestamp IS NOT NULL AND timestamp <= current_timestamp()",
        "description": "Timestamp must not be in the future"
    }
]

In [9]:
def silver_transform(df:DataFrame) -> DataFrame:

    # Clean any non-timestamp characters first
    df = df.withColumn(
        "transaction_timestamp", 
        F.regexp_replace(F.col("timestamp"), "[^0-9\\-: ]", "")
    )

    # Use try_cast to handle invalid timestamps gracefully by returning NULL
#     df = df.withColumn(
#         "transaction_timestamp",
#         F.expr("try_cast(transaction_timestamp as timestamp)")
#     )

    # Cast to timestamp type
    df = df.withColumn("transaction_timestamp", F.col("transaction_timestamp").cast("timestamp"))

    # Remove duplicates
    df = df.dropDuplicates(subset=["transaction_id","timestamp"])

    # Standardize Data
    df = (df
            .withColumn("amount", F.abs(F.col("amount")))
            .withColumn("transaction_type", F.lower(F.col("transaction_type")))
            .withColumn("category", F.lower(F.col("category")))
            .withColumn("status", F.lower(F.col("status")))
    )

    # Filter Data
    # Address bronze layer data validation check concerns
    df = df.filter(
                    (F.col('transaction_id').isNotNull()) # transaction id must exist
                    & (F.col('amount') > 0) # amount must be positive
                    & ((F.col('transaction_timestamp') <= F.current_date()) # must be <= current date
                    |(F.col('transaction_timestamp').isNull()))# or must be Null, no future timestamps
    ) 
    
    # Split timestamp into date and time and year_month for paritioning
    df = (df
            .withColumn("transaction_date", F.to_date("transaction_timestamp"))
            .withColumn("transaction_time", F.date_format("transaction_timestamp", "HH:mm:ss"))
            .withColumn("year_month", F.date_format(F.col("transaction_date"), "yyyy-MM"))
    )
    
    # Add processing timestamp for bookkeeping
    df = (df
            .withColumn("processing_timestamp", F.current_timestamp())
    )

    return df

In [10]:
# Define silver validation rules
silver_validation_rules = [
    {
        "name": "valid_transaction_type",
        "condition": "transaction_type IN ('debit', 'credit', 'transfer', 'payment', 'withdrawal', 'deposit') OR transaction_type IS NULL",
        "description": "Transaction type must be one of the valid types"
    },
    {
        "name": "valid_status",
        "condition": "status IN ('completed', 'pending', 'failed', 'cancelled', 'refunded') OR status IS NULL",
        "description": "Status must be one of the valid statuses"
    },
    {
        "name": "valid_currency",
        "condition": "currency IS NOT NULL AND length(currency) = 3",
        "description": "Currency code should be 3 characters if present"
    },
    {
        "name": "valid_transaction_timestamp",
        "condition": "transaction_timestamp IS NOT NULL AND transaction_timestamp <= current_timestamp()",
        "description": "Transaction timestamp must not be NULL or in the future"
    }
]

In [11]:
def gold_transform(df:DataFrame) -> Dict:
    gold_dfs = {}

    # Gold aggregation 1: Daily summary by category
    daily_category = (df
        .groupBy("transaction_date", "category")
        .agg(
            F.count("transaction_id").alias("transaction_count"),
            F.sum("amount").alias("total_amount"),
            F.avg("amount").alias("avg_amount"),
            F.min("amount").alias("min_amount"),
            F.max("amount").alias("max_amount"),
            F.countDistinct("customer_id").alias("unique_customers")
        )
        .withColumn("processing_timestamp", F.current_timestamp())
    )
    
    gold_dfs["daily_category"] = daily_category
    
    # Gold aggregation 2: Customer summary
    customer_summary = (df
        .groupBy("customer_id")
        .agg(
            F.count("transaction_id").alias("transaction_count"),
            F.sum("amount").alias("total_amount"),
            F.avg("amount").alias("avg_amount"),
            F.min("transaction_date").alias("first_transaction_date"),
            F.max("transaction_date").alias("last_transaction_date"),
            F.approx_count_distinct("category").alias("category_count")
        )
        .withColumn("processing_timestamp", F.current_timestamp())
        .withColumn("days_since_last_transaction", 
                    F.datediff(F.current_date(), F.col("last_transaction_date")))
    )
    
    gold_dfs["customer_summary"] = customer_summary
    
    # Gold aggregation 3: Transaction type summary
    transaction_type_summary = (df
        .groupBy("transaction_type")
        .agg(
            F.count("transaction_id").alias("transaction_count"),
            F.sum("amount").alias("total_amount"),
            F.avg("amount").alias("avg_amount")
        )
        .withColumn("processing_timestamp", F.current_timestamp())
    )
    
    gold_dfs["transaction_type_summary"] = transaction_type_summary

    return gold_dfs

In [12]:
# Define gold validation rules
gold_validation_rules = [
    {
        "name": "positive_transaction_counts",
        "condition": "transaction_count > 0",
        "description": "Transaction counts should be positive"
    },
    {
        "name": "valid_total_amounts",
        "condition": "total_amount >= 0",
        "description": "Total amounts should not be negative"
    }
]

### Running Full Batch Pipeline

In [13]:
run_batch_de_pipeline(spark, 'csv', source_path, get_schema(), 
                        bronze_table, silver_table, gold_table, 
                        None, silver_transform, gold_transform, 
                        bronze_validation_rules, silver_validation_rules, gold_validation_rules,
                        pipeline_name='Financial_DE_Pipeline')

2025-04-09 16:10:24,228 - INFO - --Starting data pipeline execution with ID: Financial_DE_Pipeline_20250409_161024--
2025-04-09 16:10:24,230 - INFO - Starting bronze layer processing for c:\GitHub\DE_Pipelines\data\financial_transactions.csv
2025-04-09 16:10:25,258 - INFO - Successfully read CSV data from c:\GitHub\DE_Pipelines\data\financial_transactions.csv
2025-04-09 16:10:25,260 - INFO - No transformation function defined
2025-04-09 16:10:52,619 - INFO - Successfully wrote bronze table: de_pipelines.financial_osb_bronze
2025-04-09 16:10:54,272 - INFO - Running data quality checks for bronze layer
2025-04-09 16:10:55,451 - INFO - Data Quality Metrics for bronze layer:
2025-04-09 16:10:55,453 - INFO -   - Column Count: 15
2025-04-09 16:10:55,454 - INFO -   - Columns: ['transaction_id', 'timestamp', 'customer_id', 'account_number', 'transaction_type', 'amount', 'currency', 'balance_after', 'status', 'merchant', 'category', 'location', 'ingestion_timestamp', 'source_file', 'batch_id']


{'status': 'success',
 'pipeline_id': 'Financial_DE_Pipeline_20250409_161024',
 'bronze_version': 4,
 'silver_version': 14,
 'timestamp': '2025-04-09T16:12:55.834109',
 'duration_seconds': 151.60424041748047,
 'metrics': {'pipeline_id': 'Financial_DE_Pipeline_20250409_161024',
  'start_time': '2025-04-09T16:10:24.228870',
  'stages': {'bronze': {'duration_seconds': 32.226003885269165,
    'version': 4,
    'status': 'success'},
   'bronze_optimize': {'layer': 'bronze',
    'duration_seconds': 3.5109047889709473,
    'status': 'success'},
   'silver': {'duration_seconds': 25.014113187789917,
    'version': 14,
    'status': 'success',
    'source_bronze_version': 4},
   'silver_optimize': {'layer': 'silver',
    'duration_seconds': 13.371373653411865,
    'status': 'success'},
   'gold': {'duration_seconds': 69.01630115509033,
    'status': 'success',
    'source_silver_version': 14,
    'tables': ['daily_category',
     'customer_summary',
     'transaction_type_summary']},
   'gold_op

### Running Each Layer In Isolation

In [14]:
spark = initialize_local_spark_delta_lake("Financial Data Pipeline - Testing")

2025-04-09 16:12:57,083 - INFO - ---Spark session initialized with Delta Lake support---


#### Bronze

In [15]:
bronzedf, bronze_version = process_batch_bronze_layer(spark, 'csv', source_path, get_schema(), bronze_table,
                               bronze_transform=None, validation_rules=bronze_validation_rules,
                               pipeline_id='test', mode='test')

2025-04-09 16:12:58,575 - INFO - Starting bronze layer processing for c:\GitHub\DE_Pipelines\data\financial_transactions.csv
2025-04-09 16:12:58,640 - INFO - Successfully read CSV data from c:\GitHub\DE_Pipelines\data\financial_transactions.csv
2025-04-09 16:12:58,641 - INFO - No transformation function defined
2025-04-09 16:12:58,642 - WARNING - --- Bronze layer in Test Mode ---
2025-04-09 16:12:58,644 - INFO - Running data quality checks for bronze layer
2025-04-09 16:12:59,144 - INFO - Data Quality Metrics for bronze layer:
2025-04-09 16:12:59,145 - INFO -   - Column Count: 15
2025-04-09 16:12:59,147 - INFO -   - Columns: ['transaction_id', 'timestamp', 'customer_id', 'account_number', 'transaction_type', 'amount', 'currency', 'balance_after', 'status', 'merchant', 'category', 'location', 'ingestion_timestamp', 'source_file', 'batch_id']
2025-04-09 16:12:59,150 - INFO -   - Sample size: 169
2025-04-09 16:12:59,151 - WARNING -   - Columns with high null percentages: {'merchant': 46.7

In [16]:
bronzedf.show(5)

+--------------+-------------------+-----------+--------------+----------------+-------+--------+-------------+---------+----------+---------+----------------+--------------------+--------------------+--------+
|transaction_id|          timestamp|customer_id|account_number|transaction_type| amount|currency|balance_after|   status|  merchant| category|        location| ingestion_timestamp|         source_file|batch_id|
+--------------+-------------------+-----------+--------------+----------------+-------+--------+-------------+---------+----------+---------+----------------+--------------------+--------------------+--------+
|   TXN00000001|               NULL| CUST001040| ACCT-86152351|          refund| 598.65|     USD|      2900.03| disputed|Mastercard|education|      Denver, CO|2025-04-09 16:12:...|file:///c:/GitHub...|    test|
|   TXN00000002|2023-06-04 15:42:51|       NULL| ACCT-52201156|        transfer|-908.13|     USD|      5944.76|completed|      NULL|     NULL|   San Diego, 

#### Silver

In [17]:
silverdf, silver_version = process_batch_silver_layer(spark, bronze_table, silver_table, 
                                                    silver_transform=silver_transform, validation_rules=silver_validation_rules,
                                                    pipeline_id='test', mode='test', bronze_version=None)

2025-04-09 16:12:59,884 - INFO - Starting silver layer processing
2025-04-09 16:13:00,521 - INFO - Successfully read bronze data version 4
2025-04-09 16:13:00,649 - INFO - Transformation function applied
2025-04-09 16:13:00,650 - WARNING - --- Silver layer in Test Mode ---
2025-04-09 16:13:00,652 - INFO - Running data quality checks for silver layer
2025-04-09 16:13:06,653 - INFO - Data Quality Metrics for silver layer:
2025-04-09 16:13:06,654 - INFO -   - Column Count: 20
2025-04-09 16:13:06,656 - INFO -   - Columns: ['transaction_id', 'timestamp', 'customer_id', 'account_number', 'transaction_type', 'amount', 'currency', 'balance_after', 'status', 'merchant', 'category', 'location', 'ingestion_timestamp', 'source_file', 'batch_id', 'transaction_timestamp', 'transaction_date', 'transaction_time', 'year_month', 'processing_timestamp']
2025-04-09 16:13:06,658 - INFO -   - Sample size: 134
2025-04-09 16:13:06,660 - WARNING -   - Columns with high null percentages: {'merchant': 45.5223880

In [18]:
silverdf.show(5)

+--------------+-------------------+-----------+--------------+----------------+-------+--------+-------------+---------+-----------+-------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+----------------+----------+--------------------+
|transaction_id|          timestamp|customer_id|account_number|transaction_type| amount|currency|balance_after|   status|   merchant|     category|        location| ingestion_timestamp|         source_file|            batch_id|transaction_timestamp|transaction_date|transaction_time|year_month|processing_timestamp|
+--------------+-------------------+-----------+--------------+----------------+-------+--------+-------------+---------+-----------+-------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+----------------+----------+--------------------+
|   TXN00000001|               NULL| CUST001040| ACC

##### Investigating Silver Validation Check Failures:
Corrupt "status" column

In [19]:
silverdf.groupBy('status').count().show()

+---------+-----+
|   status|count|
+---------+-----+
|  pendlng|    1|
|completed|  264|
| dioputed|    1|
| dirputed|    1|
|completpd|    1|
|   failed|  285|
|     NULL|   13|
|completei|    1|
|  pjnding|    1|
| oeversed|    1|
|  sending|    1|
| disiuted|    1|
|  pendinz|    1|
| repersed|    1|
| disptted|    1|
|  pendiny|    1|
|complethd|    1|
|  pending|  269|
|   faihed|    1|
|  pedding|    1|
+---------+-----+
only showing top 20 rows



#### Gold

In [20]:
gold_dfs = process_batch_gold_layer(spark, silver_table, gold_table, gold_transform, 
                                    validation_rules=gold_validation_rules, pipeline_id='test', mode='test', silver_version=None)

2025-04-09 16:13:12,241 - INFO - Starting gold layer processing
2025-04-09 16:13:12,744 - INFO - Successfully read silver data version 15
2025-04-09 16:13:12,891 - INFO - Transformation function applied
2025-04-09 16:13:12,893 - WARNING - Gold layer in Test Mode
2025-04-09 16:13:12,894 - INFO - Running data quality checks for de_pipelines.financial_osb_gold_daily_category layer
2025-04-09 16:13:21,892 - INFO - Data Quality Metrics for de_pipelines.financial_osb_gold_daily_category layer:
2025-04-09 16:13:21,893 - INFO -   - Column Count: 9
2025-04-09 16:13:21,894 - INFO -   - Columns: ['transaction_date', 'category', 'transaction_count', 'total_amount', 'avg_amount', 'min_amount', 'max_amount', 'unique_customers', 'processing_timestamp']
2025-04-09 16:13:21,897 - INFO -   - Sample size: 198
2025-04-09 16:13:21,899 - WARNING -   - Columns with high null percentages: {'category': 30.303030303030305}
2025-04-09 16:13:21,901 - INFO - Validating dataframe with 2 rules
2025-04-09 16:13:27,05

In [21]:
for table_name in gold_dfs.keys():
    gold_dfs[table_name].show(5)

+----------------+--------+-----------------+------------------+-----------------+----------+----------+----------------+--------------------+
|transaction_date|category|transaction_count|      total_amount|       avg_amount|min_amount|max_amount|unique_customers|processing_timestamp|
+----------------+--------+-----------------+------------------+-----------------+----------+----------+----------------+--------------------+
|      2023-05-25|    fees|                1|             18.07|            18.07|     18.07|     18.07|               1|2025-04-09 16:13:...|
|      2023-12-05|  income|                1|           1818.85|          1818.85|   1818.85|   1818.85|               1|2025-04-09 16:13:...|
|      2023-10-01|    fees|                1|             32.44|            32.44|     32.44|     32.44|               1|2025-04-09 16:13:...|
|      2023-12-27|  dining|                1|           1307.27|          1307.27|   1307.27|   1307.27|               1|2025-04-09 16:13:...|